# Final Project Computer Vision
## Truth in Motion: Depth and Flow Enhanced DeepFake Detection

Authors: Aimee Lin, Neli Catar and Gellert Toth

### Imports

In [1]:
!git clone git@github.com:DepthAnything/Depth-Anything-V2.git

Cloning into 'Depth-Anything-V2'...
remote: Enumerating objects: 142, done.
remote: Total 142 (delta 0), reused 0 (delta 0), pack-reused 142 (from 1)
Receiving objects: 100% (142/142), 45.17 MiB | 6.56 MiB/s, done.
Resolving deltas: 100% (46/46), done.


In [11]:
!mkdir -p DepthAnythingV2Checkpoints
!curl -L https://huggingface.co/depth-anything/Depth-Anything-V2-Base/resolve/main/depth_anything_v2_vitb.pth?download=true --output DepthAnythingV2Checkpoints/depth_anything_v2_vitb.pth


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1140  100  1140    0     0   7116      0 --:--:-- --:--:-- --:--:--  7125
100  371M  100  371M    0     0  15.3M      0  0:00:24  0:00:24 --:--:-- 19.9M


In [14]:
!git clone git@github.com:sniklaus/pytorch-pwc.git
!mv pytorch-pwc pytorch_pwc

Cloning into 'pytorch-pwc'...
remote: Enumerating objects: 251, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 251 (delta 40), reused 62 (delta 24), pack-reused 169 (from 1)
Receiving objects: 100% (251/251), 66.96 MiB | 6.95 MiB/s, done.
Resolving deltas: 100% (120/120), done.


In [ ]:
import cv2
import glob
import numpy as np
import matplotlib.pyplot as plt
import os
import random
import re
import sys
import torch
import torch.nn as nn
import torch.nn.init as init
import torchvision.transforms as T
import importlib.util

from facenet_pytorch import MTCNN
from IPython.display import display
from matplotlib import colors
from PIL import Image, ImageShow
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from transformers import AutoModel, AutoImageProcessor
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
import time
from torch.amp import autocast, GradScaler


depth_anything_path = os.path.abspath("Depth-Anything-V2")

if depth_anything_path not in sys.path:
    sys.path.append(depth_anything_path)
from depth_anything_v2.dpt import DepthAnythingV2

argv_backup = sys.argv
sys.argv = [sys.argv[0]] 
from pytorch_pwc.run import estimate 
sys.argv = argv_backup

### Globals

In [13]:
# data folders
video_root = './data' # where the data is stored
data_root = './preprocessed_data' # where the pre processed data gets stored
data_flow = "./flow_data" # where flow data gets saved
data_depth = "./depth_data"

# data loading
BATCH_SIZE = 64 # batch size of the preprocessed data

# device
DEVICE = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
print(DEVICE)
# training
num_epochs = 5 # epochs when training
SKIP_FRAMES = 2 # When processing the video jump some frames to reduce data size


cuda


### Utils

In [ ]:
def split_video_into_frames(path, save_path):
    cap = cv2.VideoCapture(path)

    frames = []
    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break
        frames.append(frame)

    cap.release()

    faces = [frame_faces for i in range(0, len(frames), BATCH_SIZE) for frame_faces in mtcnn(frames[i:min(len(frames),i+BATCH_SIZE)])]

    os.makedirs(save_path, exist_ok=True)
    for i, frame_faces in enumerate(faces):
        if frame_faces is None:
            continue
        face = frame_faces[0]
        img_pil = Image.fromarray(face.permute(1, 2, 0).numpy().astype(np.uint8))
        img_pil.save(os.path.join(save_path, f"frame_{i}.png"))

def list_leaf_folders(root_dir):
    leaf_folders = []
    for dirpath, dirnames, _ in os.walk(root_dir):
        if not dirnames:
            leaf_folders.append(dirpath)
    return leaf_folders

def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower() for text in re.split(r'(\d+)', s)]

def list_sorted_images(folder, extensions={'.png', '.jpg', '.jpeg', '.bmp'}):
    files = [f for f in os.listdir(folder)
             if os.path.isfile(os.path.join(folder, f)) and os.path.splitext(f)[1].lower() in extensions]
    files.sort(key=natural_sort_key)
    return [os.path.join(folder, f) for f in files]

def flow_to_rgb(flow):
    u = flow[..., 0]
    v = flow[..., 1] 

    magnitude = np.sqrt(u**2 + v**2) 
    direction = np.arctan2(v, u)
    mag_max = np.max(magnitude)
    magnitude = magnitude / mag_max if mag_max != 0 else magnitude

    direction = (direction + np.pi) / (2 * np.pi)
    saturation = magnitude
    value = np.ones_like(magnitude)
    hsv_image = np.stack((direction, saturation, value), axis=-1)
    rgb_image = colors.hsv_to_rgb(hsv_image.astype(np.float32))
    return rgb_image

def calculate_flow_picture(img1, img2):
    res = estimate(img1, img2)
    res = np.array(res.numpy(force=True).transpose(1, 2, 0), np.float32)
    res = flow_to_rgb(res)
    return res

def calculate_depth_picture(model, img):
    res = model.infer_image(img)
    depth_min = np.min(res)
    depth_max = np.max(res)
    if depth_max - depth_min < 1e-6:
        normalised = np.zeros_like(res, dtype=np.uint8)
    else:
        normalised = 255 * (res - depth_min) / (depth_max - depth_min)
        normalised = normalised.astype(np.uint8)
    return normalised

def collect_video_folders(base_dir):
    folders = []
    for root, dirs, _ in os.walk(base_dir):
        for d in dirs:
            folder_path = os.path.join(root, d)
            # check if it contains frames
            if any(fname.lower().endswith(('.jpg', '.png')) for fname in os.listdir(folder_path)):
                folders.append(folder_path)
    return sorted(folders)

def extract_frame_paths_and_labels(base_dirs, label, train_ratio=0.8):
    train_paths = []
    test_paths = []

    for base_dir in base_dirs:
        video_folders = collect_video_folders(base_dir)
        total_videos = len(video_folders)

        train_cutoff = int(total_videos * train_ratio)

        for i, video_folder in enumerate(video_folders):
            frame_paths = glob.glob(os.path.join(video_folder, '*'))
            frame_paths = [f for f in frame_paths if f.lower().endswith(('.jpg', '.png'))]

            labeled = [(fp, label) for fp in frame_paths]

            if i < train_cutoff:
                train_paths.extend(labeled)
            else:
                test_paths.extend(labeled)

    return train_paths, test_paths

class FrameDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_path, label = self.data[idx]
        image = Image.open(image_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        label = torch.tensor(label, dtype=torch.float32) 
        return image, label
    
def repeat_channels(x):
    if isinstance(x, torch.Tensor) and x.dim() == 3 and x.shape[0] == 1:
        return x.repeat(3, 1, 1)
    elif isinstance(x, torch.Tensor) and x.dim() == 2:
        # Add channel dimension if missing
        return x.unsqueeze(0).repeat(3, 1, 1)
    else:
        raise ValueError(f"Unexpected input shape for repeat_channels: {x.shape}")

image_transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),    
    transforms.Normalize(
        mean=[0.5, 0.5, 0.5], 
        std=[0.5, 0.5, 0.5]
    )
])


def train(model, dataloader, criterion, optimizer, device, scale=True):
    if scale:
        scaler = GradScaler()
        
    model.train()
    running_loss = 0.0
    total = 0
    correct = 0
    all_labels = []
    all_probs = []
    for images, labels in tqdm(dataloader):
        images = images.to(device)
        labels = labels.to(device).unsqueeze(1)
        optimizer.zero_grad()
        if scale:
            with autocast(device_type="cuda", dtype=torch.float16):
                outputs = model(images)
                loss = criterion(outputs, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:


            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        
        running_loss += loss.item() * images.size(0)
        
        probs = torch.sigmoid(outputs)
        preds = probs > 0.5
        correct += (preds == labels.bool()).sum().item()
        total += labels.size(0)
        
        all_labels.append(labels.cpu())
        all_probs.append(probs.cpu())
        
    epoch_loss = running_loss / total
    epoch_acc = correct / total
    all_labels = torch.cat(all_labels).numpy()
    all_probs = torch.cat(all_probs).detach().numpy()
    return epoch_loss, epoch_acc, all_labels, all_probs

def evaluate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    total = 0
    correct = 0
    all_labels = []
    all_probs = []
    
    with torch.no_grad():
        for images, labels in tqdm(dataloader):
            images = images.to(device)
            labels = labels.to(device).unsqueeze(1)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * images.size(0)
            
            probs = torch.sigmoid(outputs)
            preds = probs > 0.5
            correct += (preds == labels.bool()).sum().item()
            total += labels.size(0)
            
            all_labels.append(labels.cpu())
            all_probs.append(probs.cpu())
    
    epoch_loss = running_loss / total
    epoch_acc = correct / total
    all_labels = torch.cat(all_labels).numpy()
    all_probs = torch.cat(all_probs).numpy()
    return epoch_loss, epoch_acc, all_labels, all_probs

def plot_training_progress(train_accuracies, val_accuracies, train_aurocs, val_aurocs):
    epochs = range(1, len(train_accuracies)+1)
    plt.figure(figsize=(12,5))

    plt.subplot(1,2,1)
    plt.plot(epochs, train_accuracies, label='Train Acc')
    plt.plot(epochs, val_accuracies, label='Val Acc')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.title('Accuracy')
    plt.xticks(list(epochs))
    plt.ylim(0.5, 1.0)

    plt.subplot(1,2,2)
    plt.plot(epochs, train_aurocs, label='Train AUROC')
    plt.plot(epochs, val_aurocs, label='Val AUROC')
    plt.xlabel('Epoch')
    plt.ylabel('AUROC')
    plt.legend()
    plt.title('AUROC')
    plt.xticks(list(epochs))
    plt.ylim(0.5, 1.0)

    plt.tight_layout()
    plt.show()

def train_fp16_distilattion(model_learner, model_teacher, dataloader, criterion, optimizer, device):
    scaler = GradScaler(device)
    model_learner.train()
    model_teacher.train()
    all_labels = []
    all_probs = []
    running_loss = 0.0
    total = 0
    correct = 0

    for images, labels in tqdm.tqdm(dataloader):
        images = images.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True).unsqueeze(1)

        optimizer.zero_grad()
        with torch.no_grad():
            with autocast(device_type="cuda", dtype=torch.float16):
                cls_token_teacher = model_teacher(images)
                cls_token_teacher = cls_token_teacher.detach()
        with autocast(device_type="cuda", dtype=torch.float16):
            cls_token_teacher_projected = model_learner.cls_projector(cls_token_teacher)
            cls_token, outputs  = model_learner(images)

            loss = criterion(outputs, labels, cls_token, cls_token_teacher_projected)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item() * images.size(0)

        probs = torch.sigmoid(outputs)
        preds = probs > 0.5
        correct += (preds == labels.bool()).sum().item()
        total += labels.size(0)
        
        all_labels.append(labels.cpu())
        all_probs.append(probs.cpu())

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    all_labels = torch.cat(all_labels).numpy()
    all_probs = torch.cat(all_probs).numpy()
    return epoch_loss, epoch_acc, all_labels, all_probs

def evaluate_fp16_distilattion(model_learner, model_teacher, dataloader, criterion, device):
    model_teacher.eval()
    model_learner.eval()
    all_labels, all_probs = [], []
    running_loss = 0.0
    total = 0
    correct = 0

    with torch.no_grad():
        for images, labels in tqdm.tqdm(dataloader):
            images = images.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True).unsqueeze(1)

            with autocast(device_type="cuda", dtype=torch.float16):
                cls_token_teacher = model_teacher(images)
                cls_token_teacher = cls_token_teacher.detach()
                cls_token_teacher_projected = model_learner.cls_projector(cls_token_teacher)
                cls_token, outputs = model_learner(images)

                loss = criterion(outputs, labels, cls_token, cls_token_teacher_projected)

            running_loss += loss.item() * images.size(0)
            probs = torch.sigmoid(outputs)
            preds = probs > 0.5
            correct += (preds == labels.bool()).sum().item()
            total += labels.size(0)
            
            all_labels.append(labels.cpu())
            all_probs.append(probs.cpu())

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    all_labels = torch.cat(all_labels).numpy()
    all_probs = torch.cat(all_probs).numpy()
    return epoch_loss, epoch_acc, all_labels, all_probs

class DistillationLoss(nn.Module):
    def __init__(self, alpha=0.5, temperature=1.0):
        super().__init__()
        self.alpha = alpha
        self.temperature = temperature
        self.bce = nn.BCEWithLogitsLoss()
        self.distill_loss = self._kl_loss

    def forward(self, student_output, labels, student_cls, teacher_cls):
        cls_loss = self.bce(student_output, labels)
        distill = self.distill_loss(student_cls, teacher_cls)
        total_loss = self.alpha * cls_loss + (1 - self.alpha) * distill
        return total_loss

    def _kl_loss(self, student_cls, teacher_cls):
        T = self.temperature
        student_log_probs = F.log_softmax(student_cls / T, dim=-1)
        teacher_probs = F.softmax(teacher_cls / T, dim=-1)
        return F.kl_div(student_log_probs, teacher_probs, reduction='batchmean') * (T * T)

#### Feature extraction generation

In the sections below we tested the feature extraction generation techniques

### Data

Code for downloading the data is based on FaceForensics++ dataset and is available on https://github.com/ondyari/FaceForensics

1. Splitting the videos into frames

In [ ]:
mtcnn = MTCNN(margin=20, keep_all=True, post_process=False, device=DEVICE)

for dirpath, _, filenames in os.walk(video_root):
    if len(filenames) == 0:
        continue
    print(f"Processing videos in {dirpath}")
    for file in tqdm(filenames):
        if file.endswith('.mp4'):
            video_path = os.path.join(dirpath, file)
            relative_path = os.path.relpath(video_path, video_root)
            output_folder = os.path.join(data_root, os.path.splitext(relative_path)[0])
            split_video_into_frames(video_path, output_folder)



2. Calculating Optical Flow

In [ ]:
leaf_folders = list_leaf_folders(data_root)
print(leaf_folders)
for folder in tqdm(leaf_folders):
    relative_path = os.path.relpath(folder, data_root)
    images = list_sorted_images(folder)
    images = [torch.FloatTensor(np.ascontiguousarray(np.array(Image.open(image))[:, :, ::-1].transpose(2, 0, 1).astype(np.float32) * (1.0 / 255.0))) for image in images]
    for i in range(0, len(images)-1, SKIP_FRAMES):
        img1, img2 = images[i], images[i+1]
        res = calculate_flow_picture(img1, img2)
        save_path = os.path.join(data_flow, relative_path)
        os.makedirs(save_path, exist_ok=True)
        rgb_flow_uint8 = (res * 255).astype(np.uint8)
        pil_img = Image.fromarray(rgb_flow_uint8, mode='RGB')
        pil_img.save(os.path.join(save_path, f"flow_image_{i}.png"))

3. Depth Estimation

In [ ]:
model_configs = {
    'vits': {'encoder': 'vits', 'features': 64, 'out_channels': [48, 96, 192, 384]},
    'vitb': {'encoder': 'vitb', 'features': 128, 'out_channels': [96, 192, 384, 768]},
    'vitl': {'encoder': 'vitl', 'features': 256, 'out_channels': [256, 512, 1024, 1024]},
    'vitg': {'encoder': 'vitg', 'features': 384, 'out_channels': [1536, 1536, 1536, 1536]}
}

model = DepthAnythingV2(**model_configs['vitb'])
model.load_state_dict(torch.load(f'DepthAnythingV2Checkpoints/depth_anything_v2_vitb.pth', map_location='cpu'))
model = model.to(DEVICE).eval()


leaf_folders = list_leaf_folders(data_root)
for folder in tqdm(leaf_folders):
    relative_path = os.path.relpath(folder, data_root)
    images = list_sorted_images(folder)
    for i in range(0, len(images), SKIP_FRAMES):
        img_path = images[i]
        img = np.array(Image.open(img_path))
        res = calculate_depth_picture(model, img)
        save_path = os.path.join(data_depth, relative_path)
        os.makedirs(save_path, exist_ok=True)
        pil_img = Image.fromarray(res, mode='L')  
        pil_img.save(os.path.join(save_path, f"depth_image_{i}.png"))

 68%|██████▊   | 270/400 [2:06:04<1:27:07, 40.21s/it]

### Network

In [ ]:
class DinoBinaryClassifier(nn.Module):
    def __init__(self, backbone):
        super().__init__()
        self.backbone = backbone
        self.classifier = nn.Sequential(
            nn.Linear(self.backbone.config.hidden_size, 256, bias=True),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 1, bias=True)  # binary output
        )
        self._init_weights()

    def _init_weights(self):
        for layer in self.classifier:
            if isinstance(layer, nn.Linear):
                init.kaiming_normal_(layer.weight, nonlinearity='relu')
                init.zeros_(layer.bias)

    def forward(self, pixel_values):
        outputs = self.backbone(pixel_values=pixel_values)
        cls_token = outputs.last_hidden_state[:, 0]  
        return self.classifier(cls_token)

#### Flow based network 

Preparing the dataloader

In [ ]:
# Define your paths
original_sources = [
    'flow_data/original_sequences/actors/c23/videos',
    'flow_data/original_sequences/youtube/c23/videos'
]

manipulated_sources = [
    'flow_data/manipulated_sequences/DeepFakeDetection/c23/videos',
    'flow_data/manipulated_sequences/Deepfakes/c23/videos',
    # 'flow_data/manipulated_sequences/Face2Face/c23/videos',
    # 'flow_data/manipulated_sequences/FaceShifter/c23/videos',
    # 'flow_data/manipulated_sequences/FaceSwap/c23/videos',
    # 'flow_data/manipulated_sequences/NeuralTextures/c23/videos'
]

# Collect data
train_real, test_real = extract_frame_paths_and_labels(original_sources, label=0)
train_fake, test_fake = extract_frame_paths_and_labels(manipulated_sources, label=1)

# Combine
train_data = train_real + train_fake
test_data = test_real + test_fake

# Sample output
print(f"Training samples: {len(train_data)}")
print(f"Testing samples: {len(test_data)}")
print(f"Sample training item: {train_data[0]}")

train_dataset = FrameDataset(train_data, transform=image_transform)
test_dataset = FrameDataset(test_data, transform=image_transform)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=4, pin_memory=True)

In [ ]:
model_name = "facebook/dinov2-base"
processor = AutoImageProcessor.from_pretrained(model_name)
backbone = AutoModel.from_pretrained(model_name)

for param in backbone.parameters():
    param.requires_grad = False

for name, param in backbone.named_parameters():
    if "encoder.layer.10" in name or "encoder.layer.11" in name:
        param.requires_grad = True

criterion = torch.nn.BCEWithLogitsLoss()
backbone_params = []
classifier_params = []

for name, param in model.named_parameters():
    if param.requires_grad:
        if "backbone" in name:
            backbone_params.append(param)
        else:
            classifier_params.append(param)
print(len(backbone_params), len(classifier_params))
optimizer = torch.optim.Adam([
    {"params": backbone_params, "lr": 1e-5},
    {"params": classifier_params, "lr": 1e-4},
])

model = DinoBinaryClassifier(backbone)
model.to(DEVICE)

In [ ]:
train_accuracies, val_accuracies = [], []
train_aurocs, val_aurocs = [], []
epoch_times = []

start_epoch = 1
num_epochs = 5
total_start = time.time()

for epoch in range(start_epoch, start_epoch + num_epochs):
    # save_path = f"depth_swin_epoch_{epoch}.pth"
    epoch_start = time.time()
    
    train_loss, train_acc, train_labels, train_probs = train(model, train_loader, criterion, optimizer, DEVICE)
    val_loss, val_acc, val_labels, val_probs = evaluate(model, test_loader, criterion, DEVICE)
    
    # AUROC
    train_auroc = roc_auc_score(train_labels, train_probs)
    val_auroc = roc_auc_score(val_labels, val_probs)
    
    train_accuracies.append(train_acc)
    val_accuracies.append(val_acc)
    train_aurocs.append(train_auroc)
    val_aurocs.append(val_auroc)
    
    epoch_time = time.time() - epoch_start
    epoch_times.append(epoch_time)
    
    print(f"Epoch {epoch}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Train AUROC: {train_auroc:.4f}")
    print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f} | Val AUROC: {val_auroc:.4f}")
    print(f"Epoch time: {epoch_time:.2f} seconds")
    
    # torch.save({
    #     'epoch': epoch,
    #     'model_state_dict': model.state_dict(),
    #     'optimizer_state_dict': optimizer.state_dict(),
    #     'train_loss': train_loss,
    #     'val_loss': val_loss,
    # }, save_path)

In [ ]:
plot_training_progress(train_accuracies, val_accuracies, train_aurocs, val_aurocs)
save_path = '/checkpoints/flow_based_classifier_dinov2_base.pth'
torch.save(model.state_dict(), save_path)

#### Depth Based network

In [ ]:
# Define your paths
original_sources = [
    'depth_data/original_sequences/actors/c23/videos',
    'depth_data/original_sequences/youtube/c23/videos'
]

manipulated_sources = [
    'depth_data/manipulated_sequences/DeepFakeDetection/c23/videos',
    'depth_data/manipulated_sequences/Deepfakes/c23/videos',
    # 'depth_data/manipulated_sequences/Face2Face/c23/videos',
    # 'depth_data/manipulated_sequences/FaceShifter/c23/videos',
    # 'depth_data/manipulated_sequences/FaceSwap/c23/videos',
    # 'depth_data/manipulated_sequences/NeuralTextures/c23/videos'
]

# Collect data
train_real, test_real = extract_frame_paths_and_labels(original_sources, label=0)
train_fake, test_fake = extract_frame_paths_and_labels(manipulated_sources, label=1)

# Combine
train_data = train_real + train_fake
test_data = test_real + test_fake

# Sample output
print(f"Training samples: {len(train_data)}")
print(f"Testing samples: {len(test_data)}")
print(f"Sample training item: {train_data[0]}")

train_dataset = FrameDataset(train_data, transform=image_transform)
test_dataset = FrameDataset(test_data, transform=image_transform)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=4, pin_memory=True)

In [ ]:
model_name = "facebook/dinov2-base"
processor = AutoImageProcessor.from_pretrained(model_name)
backbone = AutoModel.from_pretrained(model_name)

for param in backbone.parameters():
    param.requires_grad = False

for name, param in backbone.named_parameters():
    if "encoder.layer.10" in name or "encoder.layer.11" in name:
        param.requires_grad = True

criterion = torch.nn.BCEWithLogitsLoss()
backbone_params = []
classifier_params = []

for name, param in model.named_parameters():
    if param.requires_grad:
        if "backbone" in name:
            backbone_params.append(param)
        else:
            classifier_params.append(param)
print(len(backbone_params), len(classifier_params))
optimizer = torch.optim.Adam([
    {"params": backbone_params, "lr": 1e-5},
    {"params": classifier_params, "lr": 1e-4},
])

model = DinoBinaryClassifier(backbone)
model.to(DEVICE)

In [ ]:
train_accuracies, val_accuracies = [], []
train_aurocs, val_aurocs = [], []
epoch_times = []

start_epoch = 1
num_epochs = 5
total_start = time.time()

for epoch in range(start_epoch, start_epoch + num_epochs):
    # save_path = f"depth_swin_epoch_{epoch}.pth"
    epoch_start = time.time()
    
    train_loss, train_acc, train_labels, train_probs = train(model, train_loader, criterion, optimizer, DEVICE)
    val_loss, val_acc, val_labels, val_probs = evaluate(model, test_loader, criterion, DEVICE)
    
    # AUROC
    train_auroc = roc_auc_score(train_labels, train_probs)
    val_auroc = roc_auc_score(val_labels, val_probs)
    
    train_accuracies.append(train_acc)
    val_accuracies.append(val_acc)
    train_aurocs.append(train_auroc)
    val_aurocs.append(val_auroc)
    
    epoch_time = time.time() - epoch_start
    epoch_times.append(epoch_time)
    
    print(f"Epoch {epoch}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Train AUROC: {train_auroc:.4f}")
    print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f} | Val AUROC: {val_auroc:.4f}")
    print(f"Epoch time: {epoch_time:.2f} seconds")
    
    # torch.save({
    #     'epoch': epoch,
    #     'model_state_dict': model.state_dict(),
    #     'optimizer_state_dict': optimizer.state_dict(),
    #     'train_loss': train_loss,
    #     'val_loss': val_loss,
    # }, save_path)

In [ ]:
plot_training_progress(train_accuracies, val_accuracies, train_aurocs, val_aurocs)
save_path = '/checkpoints/depth_based_classifier_dinov2_base.pth'
torch.save(model.state_dict(), save_path)

### Compression methods

1. Quantization

2. Distillation

In [ ]:
class DinoBinaryClassifierForDistilattion(DinoBinaryClassifier):
    def __init__(self, backbone):
        super().__init__(backbone)
        self.cls_projector = nn.Linear(1024, 384)

    def forward(self, pixel_values):
        outputs = self.backbone(pixel_values=pixel_values)
        cls_token = outputs.last_hidden_state[:, 0]
        return cls_token, self.classifier(cls_token)


class DinoBinaryClassifierForDistilattionTeacher(DinoBinaryClassifier):
    def __init__(self, backbone):
        super().__init__(backbone)

    def forward(self, pixel_values):
        outputs = self.backbone(pixel_values=pixel_values)
        cls_token = outputs.last_hidden_state[:, 0]
        return cls_token

In [ ]:
large_model_name = "facebook/dinov2-large"
small_model_name = "facebook/dinov2-small"

backbone_large = AutoModel.from_pretrained(large_model_name)
backbone_small = AutoModel.from_pretrained(small_model_name)

for param in backbone_small.parameters():
    param.requires_grad = False

for name, param in backbone_small.named_parameters():
    if "encoder.layer.11" in name or "encoder.layer.10" in name:
        param.requires_grad = True

model_teacher = DinoBinaryClassifierForDistilattionTeacher(backbone_large)
model_teacher.load_state_dict(torch.load('/checkpoints/depth_based_classifier_dinov2_base.pth'))
model_teacher.to(DEVICE)

model_learner = DinoBinaryClassifierForDistilattion(backbone_small)
model_learner.to(DEVICE)

criterion = DistillationLoss()
backbone_params = []
classifier_params = []

for name, param in model_learner.named_parameters():
    if param.requires_grad:
        if "backbone" in name:
            backbone_params.append(param)
        else:
            classifier_params.append(param)
print(len(backbone_params), len(classifier_params))
optimizer = torch.optim.Adam([
    {"params": backbone_params, "lr": 1e-5},
    {"params": classifier_params, "lr": 1e-4},
])

In [ ]:
train_accuracies, val_accuracies = [], []
train_aurocs, val_aurocs = [], []
epoch_times = []

start_epoch = 1
num_epochs = 5
total_start = time.time()

for epoch in range(start_epoch, start_epoch + num_epochs):
    # save_path = f"depth_swin_epoch_{epoch}.pth"
    epoch_start = time.time()
    
    train_loss, train_acc, train_labels, train_probs = train_fp16_distilattion(model_learner, model_teacher, train_loader, criterion, optimizer, DEVICE)
    val_loss, val_acc, val_labels, val_probs = evaluate_fp16_distilattion(model_learner, model_teacher, test_loader, criterion, DEVICE)
    
    # AUROC
    train_auroc = roc_auc_score(train_labels, train_probs)
    val_auroc = roc_auc_score(val_labels, val_probs)
    
    train_accuracies.append(train_acc)
    val_accuracies.append(val_acc)
    train_aurocs.append(train_auroc)
    val_aurocs.append(val_auroc)
    
    epoch_time = time.time() - epoch_start
    epoch_times.append(epoch_time)
    
    print(f"Epoch {epoch}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Train AUROC: {train_auroc:.4f}")
    print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f} | Val AUROC: {val_auroc:.4f}")
    print(f"Epoch time: {epoch_time:.2f} seconds")
    
    # torch.save({
    #     'epoch': epoch,
    #     'model_state_dict': model.state_dict(),
    #     'optimizer_state_dict': optimizer.state_dict(),
    #     'train_loss': train_loss,
    #     'val_loss': val_loss,
    # }, save_path)

In [ ]:
plot_training_progress(train_accuracies, val_accuracies, train_aurocs, val_aurocs)
save_path = '/checkpoints/flow_based_classifier_dinov2_small_distilled.pth'
torch.save(model_learner.state_dict(), save_path)